<a href="https://colab.research.google.com/github/rafaborneo/graph_course/blob/main/Borneo_Graphs_Practico1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico 1

En este práctico, correspondiente a la primera mitad de la materia, trabajaremos con un dataset extraído de Twitter. La idea es emplear los conceptos de grafos vistos en clase sobre un caso real de actualidad.

## Dataset

El dataset consiste en un conjunto de hilos de tweets, con un total de ~150000 tweets, extraídos entre Enero y Marzo de 2021. La temática de los mismos está referida a la vacunación contra el covid-19 en Argentina.

Pueden descargar el dataset del siguiente [link](https://drive.google.com/file/d/1X_qKsE8muAnom2tDX4sLlmBAO0Ikfe_G/view?usp=sharing).

### Campos

- **created_at:** Fecha del tweet
- **id_str:** ID del tweet
- **full_text:** Contenido del tweet
- **in_reply_to_status_id:** ID del tweet inmediatamente anterior en el hilo
- **in_reply_to_user_id:** Autor del tweet inmediatamente anterior en el hilo
- **user.id:** Autor del tweet
- **user_retweeters:** Lista de ID de usuarios que retweetearon el tweet
- **sentiment:** Etiquetado manual que indica el sentimiento o intención del tweet con respecto al tweet anterior en el hilo

## Configuración inicial

In [2]:
!pip install igraph

     |████████████████████████████████| 3.2 MB 5.2 MB/s 


In [3]:
#IMPORTAR LIBRERIAS REQUERIDAS PARA EL ANALISIS
import numpy as np
import pandas as pd
import networkx as nx
import igraph as ig
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_context('talk')
#from pathlib import Path

## Descargar el csv con los datos en este directorio
#DATA_DIR = Path('C:\Users\Rafael\Documents\AREA TRABAJO\DIPLODATOS 2021\7_opt_Analisis_Grafos\clases\Practico')
#INPUT_FILE = DATA_DIR / 'vacunas.csv'

## Creamos el directorio en caso de que no exista
#DATA_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
import igraph
print(igraph.__version__)

0.9.8


### Cargamos el dataset

In [11]:
dtypes = {
    'id_str': str,
    'full_text': str,
    'in_reply_to_status_id': str,
    'in_reply_to_user_id': str,
    'user.id': str
}
df = pd.read_csv("vacunas.csv", dtype=dtypes).dropna(subset=['user_retweeters'])
df['user_retweeters'] = df['user_retweeters'].apply(lambda x: [str(elem) for elem in eval(x)])
print(df.shape)
df.head()

(155123, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,"[2258074658, 159909978, 105301854, 290671142, ...",NaN
1,Sat Feb 20 03:19:59 +0000 2021,1362965193509265417,@Clon_43 @norabar Nora estaba indignada porque...,1362962469749153792,2737379453,32718111,[],NaN
2,Mon Feb 22 23:55:08 +0000 2021,1364000806740111363,"Bueno, Alberto dijo Salud o Economía. La salud...",NaN,NaN,252168075,"[1238117630696972289, 37232479, 12792246571247...",NaN
3,Tue Feb 23 00:09:14 +0000 2021,1364004354374696963,@spitta1969 Tuit del mes Spitta,1364000806740111363,252168075,1156346340802224128,[],NaN
4,Tue Feb 23 00:00:17 +0000 2021,1364002100364128260,@spitta1969 Estas onfire,1364000806740111363,252168075,153663816,[],NaN


### Observamos algunos ejemplos

In [12]:
idx = 0
print('Texto:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Texto: Seguimos esperando el comunicado de @norabar repudiando la situación respecto del gobierno y el tema vacunas. Seamos pacientes que con esto de la pandemia anda con mucho "laburo".
Retweets: 9


In [13]:
idx = 376
print('Text:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Text: Todo lo que hay que entender sobre la decisión –o no– de poner más vacunas en más brazos (por ejemplo, usar las 1º dosis en muchos y si es necesario retrasar la 2º) está en esta excelente nota de Nora Bär. https://t.co/A0I03DyxgO
Retweets: 48


### Calculamos la cantidad de hilos

In [14]:
roots = df[df['in_reply_to_user_id'].isna()]
roots.shape

(3174, 8)

## Actividades

### **1. Construcción del grafo** 

Construir el **grafo de retweets**, definido de la siguiente manera:

- Tipo de grafo: Dirigido
- Nodos: ID de los usuarios
- Enlaces: (Usuario A) ---> (Usuario B) si B retweeteó algún tweet de A

Con estos datos, el grafo debería tener alrededor de 40000 nodos y 90000 enlaces.

Considerar la versión no dirigida del grafo y estudiar su conectividad. Si existe una única "componente gigante", realizar el resto de las actividades sobre ella, en lugar de sobre el grafo completo.

Calcular las siguientes métricas globales del grafo:

- Grado medio
- Asortatividad
- Transitividad
- Coeficiente de clustering de Watts-Strogatz

**Opcional:** Comparar las métricas calculadas anteriormente con las de un grafo aleatorio con la misma distribución de grado. Pueden utilizar para ello este [método](https://networkx.org/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html?highlight=configuration#networkx.generators.degree_seq.configuration_model). Con esto en mente, comentar si los valores obtenidos anteriormente difieren significativamente del caso aleatorio.


### **2. Centralidad**

Calcular 5 métricas de centralidad de nodos. Graficar la distribución de cada una de ellas ¿Existe alguna correlación entre las distintas centralidades? 

Hacer un ranking con los 10 nodos más centrales para cada métrica. ¿Hay coincidencia entre los rankings?. ¿Qué características tienen los usuarios más centrales y sus respectivos tweets?

**Opcional:** Determinar si existe alguna correlación entre la centralidad de un nodo y su actividad en red social. Es decir, evaluar si los usuarios que más escriben son los más centrales o no.

### **3. Comunidades**

Utilizar el algoritmo de Louvain con el parámetro "resolución" igual a 1. Caracterizar las comunidades halladas (cantidad, distribución de tamaños). Utilizar la modularidad y otras dos métricas a elección para evaluar la calidad de la partición encontrada. 

Variar el parámetro "resolución" y observar cómo cambia la distribución de comunidades encontradas. ¿Existe algún valor para el cual se identifiquen dos grandes comunidades?

Elegir otro algoritmo de detección de comunidades y comparar los resultados con los obtenidos anteriormente.

**Opcional:** Correr el algoritmo de Louvain con distintas semillas aleatorias. Utilizar alguna métrica de comparación externa entre las particiones obtenidas para determinar en qué medida depende el algoritmo de la condición inicial.

In [15]:
#expansion columna user_retweeters
df1= df.explode("user_retweeters", ignore_index=True)
print(df1.shape)
df1.head()

(320948, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,2258074658,NaN
1,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,159909978,NaN
2,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,105301854,NaN
3,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,290671142,NaN
4,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,208721168,NaN


In [16]:
df1 = df1.dropna(subset=['user.id','user_retweeters'])
print(df1.shape)
df1.head()

(187297, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,2258074658,NaN
1,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,159909978,NaN
2,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,105301854,NaN
3,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,290671142,NaN
4,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,208721168,NaN


In [17]:
dir_G =  nx.from_pandas_edgelist(df1, source='user.id', target='user_retweeters', edge_attr=None, create_using=None, edge_key=None)

In [18]:
print(dir_G)

Graph with 39800 nodes and 93404 edges


In [19]:
#METRICAS GLOBALES CON NETWORKX

In [20]:
print(nx.info(dir_G))

Graph with 39800 nodes and 93404 edges


In [21]:
#lista de grados del grafo
grados = [val for (node, val) in dir_G.degree()]


In [22]:
#grado medio:promedio de los grados de cada nodo
grado_medio= sum(grados)/len(grados)
print(grado_medio)

4.693668341708543


In [23]:
#ASORTATIVIDAD BASADA EN EL GRADO; DEGREE CORRELATION; DEGREE ASSORTATIVITY
r = nx.degree_assortativity_coefficient(dir_G)
print(r)
#el valor negativo indica  que la red es DISORTATIVA


-0.2213190254294167


In [24]:
#Construimos un DataFrame de pandas para almacenar la estadística de los nodos
df3 = pd.DataFrame(index=list(dir_G.nodes()))
df3.shape

(39800, 0)

# TRANSITIVIDAD Coeficiente de clustering 

In [25]:
clustering_coefficient = nx.clustering(dir_G)
df3['Cws'] = list(clustering_coefficient.values())
df3

,Cws
2737379453,0.000000
2258074658,0.052632
159909978,0.000000
105301854,0.066667
290671142,0.066667
...,...
2765081259,0.000000
339254354,0.000000
346605471,0.000000
58065646,0.000000


# **CENTRALIDAD** 

In [26]:
deg_centrality = nx.degree_centrality(dir_G)


In [27]:
pr = nx.pagerank(dir_G, alpha = 0.8)

In [28]:
df3['degree_cent'] = list(deg_centrality.values())
df3['page_rank'] = list(pr.values())

In [ ]:
#NO SE PROCESARON BETWENESS Y CLOSENESS . DESPUES DE DOS HORAS NO SE HABIA OBTENIDO RESULTADOS conNETWORK.
#EN clase recomendaron usar otra libreria (igraph) 

In [ ]:
#estas fueron las metricas calculadas con networkx
df3

,Cws,degree_cent,page_rank
2737379453,0.000000,0.000226,0.000038
2258074658,0.052632,0.000477,0.000066
159909978,0.000000,0.000025,0.000008
105301854,0.066667,0.000251,0.000035
290671142,0.066667,0.000377,0.000053
...,...,...,...
2765081259,0.000000,0.000025,0.000025
339254354,0.000000,0.000151,0.000031
346605471,0.000000,0.000025,0.000009
58065646,0.000000,0.000025,0.000007


# CALCULO DE METRICAS USANDO IGRAPH (mas rapido)
primero hay que hacer el grafo en igrapgh

In [29]:
#ESTE CODIGO CONSTRUYE UN GRAFO para igraph a partir de los edges de nx
#OJO el grafo de nx se llama dir_G, el de igraph g_ig
#referencia: https://igraph.discourse.group/t/how-to-create-a-graph-from-pandas-dataframes/34/2

g_ig = ig.Graph.TupleList(dir_G.edges())
print(g_ig.summary())

IGRAPH UN-- 39800 93404 -- 
+ attr: name (v)


In [31]:
g_ig.eigenvector_centrality()

[0.0007483956965023574,
 0.009090829462966086,
 7.4903738594958756e-06,
 0.01069571203182479,
 0.01193749847254088,
 7.4903738594958705e-06,
 0.010886494164727747,
 0.007730629130635492,
 0.012811654533652003,
 0.011607657134092958,
 0.013741945194373913,
 0.0021882461054384336,
 0.0007094814450098964,
 0.7383386102242379,
 0.03020393661303347,
 0.00256643907661575,
 0.006045386426980044,
 0.003966114166222956,
 0.0007202620823540032,
 0.034380672010238644,
 0.010617728972683182,
 0.0214413688143887,
 0.018749209609753133,
 0.012138648969205948,
 0.0006255353687694879,
 0.0001734959550238154,
 0.009915707572858104,
 0.0010329638396514402,
 0.004982875276547719,
 0.009751691146770582,
 0.2625670874991851,
 0.0015221612340732018,
 0.0006900936470632207,
 0.0034278244380940813,
 0.027876968298827597,
 0.01020796752821169,
 0.022506502803920702,
 0.0029710714323446332,
 0.03953434590364355,
 0.010318776963616697,
 0.004044584153773152,
 0.013536091446101656,
 0.0005514954277683296,
 0.0206

In [ ]:
g_ig.edge_betweenness()

3. Comunidades

Utilizar el algoritmo de Louvain con el parámetro "resolución" igual a 1. Caracterizar las comunidades halladas (cantidad, distribución de tamaños). Utilizar la modularidad y otras dos métricas a elección para evaluar la calidad de la partición encontrada.

Variar el parámetro "resolución" y observar cómo cambia la distribución de comunidades encontradas. ¿Existe algún valor para el cual se identifiquen dos grandes comunidades?

Elegir otro algoritmo de detección de comunidades y comparar los resultados con los obtenidos anteriormente.

In [ ]:
resolutions = [0.1, 1, 10]
comms_dict = {}
for resolution in resolutions:
    comms = algorithms.louvain(dir_G, weight='weight', resolution=resolution, randomize=False)
    comms_dict[resolution] = comms 
plot_graph_by_parameters(comms_dict, parameter_name='Resolution');